# Environment

In [1]:
# nlp library
import spacy
# de, en nlp models
import de_core_news_sm, en_core_web_sm
# data wrangling
import pandas as pd
# translation
from google.cloud import translate_v2
# create google application credentials env variable
import os
# text to speech
from gtts import gTTS
from IPython.display import Audio
# dynamic class
import jdc

# Class

In [2]:
class LearningGerman:
    def __init__(self, text='testen'):
        self.text = text
        self.words, self.lemma, self.pos, self.details = ([] for c in range(4))
        self.table_ = pd.DataFrame()
        os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '.projdata/credentials.json'
        self.client = translate_v2.Client()
        self.language = self.client.detect_language(self.text)['language']
        if self.language == 'de':
            self.to = 'en'
        elif self.language == 'en':
            self.to = 'de'
        self.model = 'nmt'

In [3]:
%%add_to LearningGerman

@property
def table(self):
    if self.table_.empty:
        print('To create a complete vocab chart, please call nlp()!')
    else:
        return self.table_

In [4]:
%%add_to LearningGerman

@staticmethod
def drop_punct(table):
    if table['pos'].str.contains('PUNCT').any():
        table.drop(table[table.pos == 'PUNCT'].index, inplace=True)

In [5]:
%%add_to LearningGerman

def translate(self, words=False):
    if words == False:
        return self.client.translate(self.text, source_language=self.language, target_language=self.to, 
                         model=self.model, format_='text')['translatedText']
    else:
        return [self.client.translate(word, source_language=self.language, target_language=self.to, 
                         model=self.model, format_='text')['translatedText'] for word in self.words]

In [6]:
%%add_to LearningGerman

def nlp(self, visualize=False):
    if self.language == 'de':
        nlp = de_core_news_sm.load()
        column_names = ['deutsche', 'lemma', 'pos', 'details']
    elif self.language == 'en':
        nlp = en_core_web_sm.load()
        column_names = ['english', 'lemma', 'pos', 'details']
    doc = nlp(self.text)
    self.words.clear(), self.lemma.clear(), self.pos.clear, self.details.clear()
    for token in doc:
        self.words.append(token.text)
        self.lemma.append(token.lemma_.lower())
        self.pos.append(token.pos_)
        self.details.append(spacy.explain(token.tag_))
    words_table = pd.DataFrame(data=zip(self.words, self.lemma, self.pos, self.details), columns=column_names)
    words_table['translation'] = self.translate(words=True)
    self.drop_punct(words_table)
    if visualize == True:
        spacy.displacy.render(doc, style='dep')
    print(f'\033[1;33m{self.text}\033[m -> {self.translate()}')
    self.table_ = words_table
    return words_table

In [7]:
%%add_to LearningGerman

def say(self, translation=False, slowmode=False, autoplay=True):
    if translation == False:
        print(f'\033[1;33m{self.language}')
        speech = gTTS(self.text, lang=self.language, slow=slowmode)
        speech.save('.projdata/speech.mp3')
    else:
        print(f'\033[1m{self.to}')
        speech = gTTS(self.translate(), lang=self.to, slow=slowmode)
        speech.save('.projdata/speech.mp3')
    return Audio('.projdata/speech.mp3', autoplay=autoplay)

# Usage

## Constructor
```
LearningGerman(text: str) -> Instanciate the LearningGerman class by a given string
```
### Attributes
<span style="color: Chocolate;"><strong>text</strong></span>: string, german or english text  
<span style="color: Chocolate;"><strong>words</strong></span>: list, list of tokens extracted from self.text  
<span style="color: Chocolate;"><strong>table</strong></span>: pd.Dataframe, table of words with part of speech informations   
<span style="color: Chocolate;"><strong>client</strong></span>: google cloud translate API client    
<span style="color: Chocolate;"><strong>language</strong></span>: string, ISO 639-1 language code detected from self.text   
<span style="color: Chocolate;"><strong>to</strong></span>: string, ISO 639-1 target language code  
<span style="color: Chocolate;"><strong>model</strong></span>: string, google cloud translate model used  

In [8]:
lesson = LearningGerman(text='Ich liebe Kuchen')

## Text to Speech
```
say(self: LearningGerman(), translation: bool, slowmode: bool, autoplay: bool) -> Returns playable audio
```
### Args
<span style="color: CornflowerBlue;"><strong>translation</strong></span>: boolean, wether output will be <strong>original</strong> or <strong>translated</strong>   
<span style="color: CornflowerBlue;"><strong>slowmode</strong></span>: boolean, wether output will be in <strong>slowmode</strong> or <strong>not</strong>   
<span style="color: CornflowerBlue;"><strong>autoplay</strong></span>: boolean, wether the audio will be <strong>automatically played</strong> or <strong>not</strong>

In [9]:
lesson.say(translation=False, slowmode=True, autoplay=True)

de


In [10]:
lesson.say(translation=True, slowmode=False, autoplay=False)

en


## Translation
```
translate(self: LearningGerman(), words: bool) -> Returns translated text
```
### Args
<span style="color: CornflowerBlue;"><strong>words</strong></span>: boolean, if <strong>True</strong> returns a <strong>list of translated words</strong> instead   

In [11]:
lesson.translate()

'I love cake'

In [12]:
lesson.translate(words=True)

[]

## Natural language processing
```
nlp(self: LearningGerman(), words: bool) -> Returns original: translated text in a vocabulary chart
```
### Args
<span style="color: CornflowerBlue;"><strong>words</strong></span>: boolean, if <strong>True</strong> displays syntactic dependencies  

In [13]:
lesson.nlp()

Ich liebe Kuchen -> I love cake


,deutsche,lemma,pos,details,translation
0,Ich,ich,PRON,non-reflexive personal pronoun,I
1,liebe,lieb,VERB,"finite verb, full",love
2,Kuchen,kuchen,NOUN,"noun, singular or mass",cake


In [14]:
lesson.nlp(visualize=True)

Ich liebe Kuchen -> I love cake


,deutsche,lemma,pos,details,translation
0,Ich,ich,PRON,non-reflexive personal pronoun,I
1,liebe,lieb,VERB,"finite verb, full",love
2,Kuchen,kuchen,NOUN,"noun, singular or mass",cake
